In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:/Windows/Fonts/HANYGO230.ttf").get_name()
plt.rc("font",family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

import seaborn as sns

#import warnings
#warnings.filterwarnings("ignore")

In [2]:
import mglearn
from sklearn.datasets import load_breast_cancer, load_boston, load_iris, make_moons
from sklearn.datasets import load_digits, load_wine, load_diabetes, make_blobs

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler,MinMaxScaler

from sklearn.feature_extraction.text import CountVectorizer 
#단어의 수를 세주는 자연어처리에 사용하는 객체

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

import xgboost as xgb
# 방법 1. conda install -c anaconda py-xgboost 로 설치후 xgb 사용(성능 더좋음)
# 방법 2. from xgbost import XGBClassifier. XGBRegressor 사용(사용법 비슷해서 편리)
from xgboost import XGBClassifier, XGBRegressor
from xgboost import plot_importance, plot_tree

from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm import plot_importance, plot_metric, plot_tree

from sklearn.svm import SVR, SVC

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

from sklearn.cluster import KMeans

from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

import statsmodels.api as sm

from sklearn.tree import export_graphviz
import graphviz

In [4]:
df1 = pd.read_csv("Data/train/KOBIS_기간별박스오피스_2011-2012.csv", header=5)
df2 = pd.read_csv("Data/train/KOBIS_기간별박스오피스_2013-2014.csv", header=5)
df3 = pd.read_csv("Data/train/KOBIS_기간별박스오피스_2015-2016.csv", header=5)
df4 = pd.read_csv("Data/train/KOBIS_기간별박스오피스_2017-2018.csv", header=5)
df5 = pd.read_csv("Data/train/KOBIS_기간별박스오피스_2019-2020.csv", header=5)

In [59]:
#열이름 변경
cols=["순위","영화명","개봉일","매출액","점유율","누적매출액","관객수","누적관객수",
      "스크린수","상영횟수","대표국적","국적","제작사","배급사",
      "등급","장르","감독","배우"]

df=pd.concat([df1,df2,df3,df4,df5])
df.columns=cols

In [6]:
#원본 저장
df.to_csv("data.csv",encoding = 'utf-8-sig')

In [72]:
### 합계표시 된 행 제거
df.drop(df.loc[df.loc[:]["순위"]=="합계"].index, inplace=True)

KeyError: '순위'

In [73]:
#순위, 국적 제거
df.drop(["순위"],axis=1, inplace=True)
df.drop(["국적"],axis=1, inplace=True)

KeyError: "['순위'] not found in axis"

In [74]:
#인덱스 제거
df.reset_index(inplace=True)
df.drop(["index"],axis=1, inplace=True)

In [70]:
df.drop(["level_0"],axis=1, inplace=True)

KeyError: "['level_0'] not found in axis"

In [75]:
df.head()

,영화명,개봉일,매출액,점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,제작사,배급사,등급,장르,감독,배우
0,도둑들,2012-07-25,"93,664,808,500",3.50%,"93,664,808,500","12,983,178","12,983,178","1,091","155,392",한국,(주)케이퍼필름,(주)쇼박스,15세이상관람가,"액션,범죄",최동훈,"김윤석,김혜수,이정재,전지현,임달화,김해숙,오달수,김수현,이심결,증국상,서영주,장동..."
1,"광해, 왕이 된 남자",2012-09-13,"88,899,448,769",3.30%,"88,899,448,769","12,319,390","12,319,390","1,001","203,400",한국,"리얼라이즈픽쳐스(주),(주)씨제이이엔엠",(주)씨제이이엔엠,15세이상관람가,"사극,드라마",추창민,"이병헌,류승룡,한효주,장광,김인권,심은경,김명곤,서진원,장재현,정창국,조혜정,김남준..."
2,트랜스포머 3,2011-06-29,"74,840,681,500",2.80%,"74,840,681,500","7,784,743","7,784,743","1,409","124,450",미국,NaN,(주)씨제이이엔엠,12세이상관람가,"액션,어드벤처",마이클 베이,"조쉬 더하멜,존 말코비치,로지 헌팅턴-휘틀리,샤이아 라보프,패트릭 뎀시"
3,미션임파서블:고스트프로토콜,2011-12-15,"57,301,924,000",2.10%,"57,301,924,000","7,508,896","7,508,896","1,024","114,765",미국,파라마운트 픽쳐스,(주)씨제이이엔엠,15세이상관람가,액션,브래드 버드,"톰 크루즈,제레미 레너"
4,최종병기 활,2011-08-10,"55,827,861,500",2.10%,"55,827,861,500","7,470,633","7,470,633",615,"130,054",한국,"(주)디씨지플러스,(주)다세포클럽",롯데쇼핑㈜롯데엔터테인먼트,15세이상관람가,"사극,액션",김한민,"박해일,류승룡,김무열,문채원,이한위,김구택,이경영,박기웅,이승준,이재구,박노식,이다..."


In [10]:
df.set_index("영화명")
df

,영화명,개봉일,매출액,점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,제작사,배급사,등급,장르,감독,배우
0,도둑들,2012-07-25,"93,664,808,500",3.50%,"93,664,808,500","12,983,178","12,983,178","1,091","155,392",한국,(주)케이퍼필름,(주)쇼박스,15세이상관람가,"액션,범죄",최동훈,"김윤석,김혜수,이정재,전지현,임달화,김해숙,오달수,김수현,이심결,증국상,서영주,장동..."
1,"광해, 왕이 된 남자",2012-09-13,"88,899,448,769",3.30%,"88,899,448,769","12,319,390","12,319,390","1,001","203,400",한국,"리얼라이즈픽쳐스(주),(주)씨제이이엔엠",(주)씨제이이엔엠,15세이상관람가,"사극,드라마",추창민,"이병헌,류승룡,한효주,장광,김인권,심은경,김명곤,서진원,장재현,정창국,조혜정,김남준..."
2,트랜스포머 3,2011-06-29,"74,840,681,500",2.80%,"74,840,681,500","7,784,743","7,784,743","1,409","124,450",미국,NaN,(주)씨제이이엔엠,12세이상관람가,"액션,어드벤처",마이클 베이,"조쉬 더하멜,존 말코비치,로지 헌팅턴-휘틀리,샤이아 라보프,패트릭 뎀시"
3,미션임파서블:고스트프로토콜,2011-12-15,"57,301,924,000",2.10%,"57,301,924,000","7,508,896","7,508,896","1,024","114,765",미국,파라마운트 픽쳐스,(주)씨제이이엔엠,15세이상관람가,액션,브래드 버드,"톰 크루즈,제레미 레너"
4,최종병기 활,2011-08-10,"55,827,861,500",2.10%,"55,827,861,500","7,470,633","7,470,633",615,"130,054",한국,"(주)디씨지플러스,(주)다세포클럽",롯데쇼핑㈜롯데엔터테인먼트,15세이상관람가,"사극,액션",김한민,"박해일,류승룡,김무열,문채원,이한위,김구택,이경영,박기웅,이승준,이재구,박노식,이다..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8311,홈,2018-05-30,0,0.00%,"59,638,222",0,"8,257",1,4,한국,"(주)아토,큐브이미지웍스",(주)리틀빅픽쳐스,전체관람가,"가족,드라마",김종우,"이효제,임태풍,김하나,김건우,김경호,김기환,김대현,김동일,김범수,김상우,김성윤,김수..."
8312,홍의 장군,1973-10-13,0,0.00%,0,0,0,1,1,한국,합동영화(주),NaN,12세이상관람가,사극,이두용,"황해,고은아"
8313,홍하곡,NaN,0,0.00%,0,0,0,1,1,중국,NaN,NaN,NaN,기타,펑샤오닝,NaN
8314,후난에서 온 여인,NaN,0,0.00%,0,0,0,1,1,중국,NaN,NaN,NaN,드라마,사비,나인화


#### (2) 등급 세분화

In [111]:
df["등급"].value_counts()

청소년관람불가                             8743
15세이상관람가                            4915
전체관람가                               3102
12세이상관람가                            2973
고등학생이상관람가                            315
연소자관람불가                              314
12세관람가                               253
15세관람가                               246
18세관람가                               207
중학생이상관람가                             165
연소자관람가                               154
국민학생관람불가                             112
고등학생이상관람가,15세이상관람가                    54
연소자관람불가,청소년관람불가                       49
18세 미만인 자는 관람할 수 없는 등급                49
미정                                    43
15세 미만인 자는 관람할 수 없는 등급                36
미성년자관람불가                              34
15세관람가,15세이상관람가                       33
18세관람가,청소년관람불가                        31
12세이상관람가,12세관람가                       26
12세 미만인 자는 관람할 수 없는 등급                24
연소자관람가,전체관람가                          23
15세이상관람가,중학생이상관람가                     21
청소년관람불가,15세이상관람가

In [39]:
no_18 = ["18세관람가","연소자관람불가","연소자관람불가,청소년관람불가",
          "18세 미만인 자는 관람할 수 없는 등급",
          "미성년자관람불가","18세관람가,청소년관람불가"
          "청소년관람불가,고등학생이상관람가","국민학생관람불가,청소년관람불가",
          "청소년관람불가,12세관람가","고등학생이상관람가,청소년관람불가"]

In [40]:
df[df["등급"].isin(no_18)]["등급"].value_counts()

연소자관람불가                   314
18세관람가                    207
연소자관람불가,청소년관람불가            49
18세 미만인 자는 관람할 수 없는 등급     49
미성년자관람불가                   34
국민학생관람불가,청소년관람불가            4
청소년관람불가,12세관람가              3
고등학생이상관람가,청소년관람불가           2
Name: 등급, dtype: int64

In [38]:
df[df["등급"]=="18세관람가,청소년관람불가"]

,영화명,개봉일,매출액,점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,제작사,배급사,등급,장르,감독,배우
1310,매치 포인트,2006-04-13,"1,963,500",0.00%,"847,620,600",272,"128,056",1,13,영국,비비씨필름즈,"(주)퍼스트런,글뫼 주식회사,CJ ENM","18세관람가,청소년관람불가","멜로/로맨스,드라마",우디 알렌,"조나단 리스 마이어스,스칼렛 요한슨"
1328,피아니스트,2002-12-20,"2,070,000",0.00%,"2,604,900",265,336,1,12,프랑스,NaN,"(주)블룸즈베리리소시스리미티드,(주)엠앤드엔엔터테인먼트","18세관람가,청소년관람불가",드라마,미카엘 하네케,"이자벨 위페르,브누와 마지멜"
1372,클로저,2005-02-03,"711,000",0.00%,"1,450,069,000",237,"220,542",2,7,미국,워너 브러더스 텔레비전,"(주)스튜디오디에이치엘,(주)퍼스트런,글뫼 주식회사,소니픽쳐스릴리징코리아","18세관람가,청소년관람불가","멜로/로맨스,드라마",마이크 니콜스,"나탈리 포트만,주드 로,줄리아 로버츠,클라이브 오웬"
2421,벨벳 골드마인,1999-08-28,"504,000",0.00%,"504,000",73,73,1,1,영국,"채널4필름,골드윈필름,킬러 필름즈,미라맥스 필름즈,뉴마켓 캐피탈 그룹","(주)디스테이션,(주)엔케이컨텐츠","18세관람가,청소년관람불가","드라마,뮤지컬",토드 헤인즈,"이완 맥그리거,조나단 리스 마이어스,크리스찬 베일,토니 콜레뜨"
803,벨벳 골드마인,1999-08-28,"39,877,908",0.00%,"40,381,908","5,003","5,076",8,217,영국,"채널4필름,골드윈필름,킬러 필름즈,미라맥스 필름즈,뉴마켓 캐피탈 그룹","(주)디스테이션,(주)엔케이컨텐츠","18세관람가,청소년관람불가","드라마,뮤지컬",토드 헤인즈,"이완 맥그리거,조나단 리스 마이어스,크리스찬 베일,토니 콜레뜨"
1707,파이트 클럽,1999-11-13,"3,369,000",0.00%,"3,369,000",547,547,1,8,미국,"아트린슨프로덕션즈,폭스2000픽처스,리젠시 엔터프라이즈,타우러스 필름",(주)팝엔터테인먼트,"18세관람가,청소년관람불가","액션,범죄,드라마,스릴러",데이빗 핀처,"브래드 피트,에드워드 노튼,헬레나 본햄 카터"
1940,매치 포인트,2006-04-13,"2,220,000",0.00%,"849,840,600",358,"128,414",1,8,영국,비비씨필름즈,"(주)퍼스트런,글뫼 주식회사,CJ ENM","18세관람가,청소년관람불가","멜로/로맨스,드라마",우디 알렌,"조나단 리스 마이어스,스칼렛 요한슨"
2031,박하사탕,2000-01-01,"2,366,500",0.00%,"2,872,500",302,378,1,8,한국,"(주)이스트필름,파인하우스필름(주)","CGV아트하우스,(주)신도필림","18세관람가,청소년관람불가",드라마,이창동,"설경구,문소리,김여진,이병철,공형진,이대연,최덕문,박길수,지대한,유승목,손영순,박성..."
2205,피아니스트,2002-12-20,"1,835,500",0.00%,"4,440,400",235,571,1,7,프랑스,NaN,"(주)블룸즈베리리소시스리미티드,(주)엠앤드엔엔터테인먼트","18세관람가,청소년관람불가",드라마,미카엘 하네케,"이자벨 위페르,브누와 마지멜"
2541,2046,2004-10-15,"1,264,000",0.00%,"797,104,500",149,"123,062",1,6,홍콩,"아르떼,콜럼비아 픽쳐스 코포레이션,프랑스3시네마,젯톤프로덕션,제2 독일 텔레비젼 협...","(주)디스테이션,이십세기폭스필름코퍼레이션","18세관람가,청소년관람불가",드라마,왕가위,"양조위,공리,왕비,기무라 다쿠야,장쯔이,유가령,장첸,장만옥"


### 중복 제거

In [154]:
(df["영화명"].value_counts() >1).value_counts()

False    22310
True      4806
Name: 영화명, dtype: int64

#### (3) 장르 통일

In [151]:
## nan값 확인 후 처리
df["장르"].isnull().sum() #결측값 1421개


0

In [147]:
for i in range(len(df["장르"])):
    if type(df["장르"][i]) == float:
        df["장르"][i] = "장르없음"
    else:
        df["장르"][i] = df["장르"][i].split(",")[0]

In [148]:
df["장르"].value_counts()

드라마         9246
멜로/로맨스      4341
다큐멘터리       3272
기타          3209
애니메이션       3003
액션          2229
코미디         1842
성인물(에로)     1768
장르없음        1421
공포(호러)       893
스릴러          687
범죄           685
SF           370
공연           316
미스터리         290
어드벤처         279
전쟁           253
서부극(웨스턴)     224
판타지          215
사극           213
가족           198
뮤지컬          138
Name: 장르, dtype: int64

In [150]:
df.head()

,영화명,개봉일,매출액,점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,제작사,배급사,등급,장르,감독,배우
0,도둑들,2012-07-25,"93,664,808,500",3.50%,"93,664,808,500","12,983,178","12,983,178","1,091","155,392",한국,(주)케이퍼필름,(주)쇼박스,15세이상관람가,액션,최동훈,"김윤석,김혜수,이정재,전지현,임달화,김해숙,오달수,김수현,이심결,증국상,서영주,장동..."
1,"광해, 왕이 된 남자",2012-09-13,"88,899,448,769",3.30%,"88,899,448,769","12,319,390","12,319,390","1,001","203,400",한국,"리얼라이즈픽쳐스(주),(주)씨제이이엔엠",(주)씨제이이엔엠,15세이상관람가,사극,추창민,"이병헌,류승룡,한효주,장광,김인권,심은경,김명곤,서진원,장재현,정창국,조혜정,김남준..."
2,트랜스포머 3,2011-06-29,"74,840,681,500",2.80%,"74,840,681,500","7,784,743","7,784,743","1,409","124,450",미국,NaN,(주)씨제이이엔엠,12세이상관람가,액션,마이클 베이,"조쉬 더하멜,존 말코비치,로지 헌팅턴-휘틀리,샤이아 라보프,패트릭 뎀시"
3,미션임파서블:고스트프로토콜,2011-12-15,"57,301,924,000",2.10%,"57,301,924,000","7,508,896","7,508,896","1,024","114,765",미국,파라마운트 픽쳐스,(주)씨제이이엔엠,15세이상관람가,액션,브래드 버드,"톰 크루즈,제레미 레너"
4,최종병기 활,2011-08-10,"55,827,861,500",2.10%,"55,827,861,500","7,470,633","7,470,633",615,"130,054",한국,"(주)디씨지플러스,(주)다세포클럽",롯데쇼핑㈜롯데엔터테인먼트,15세이상관람가,사극,김한민,"박해일,류승룡,김무열,문채원,이한위,김구택,이경영,박기웅,이승준,이재구,박노식,이다..."


In [153]:
df.head()

,영화명,개봉일,매출액,점유율,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,제작사,배급사,등급,장르,감독,배우
0,도둑들,2012-07-25,"93,664,808,500",3.50%,"93,664,808,500","12,983,178","12,983,178","1,091","155,392",한국,(주)케이퍼필름,(주)쇼박스,15세이상관람가,액션,최동훈,"김윤석,김혜수,이정재,전지현,임달화,김해숙,오달수,김수현,이심결,증국상,서영주,장동..."
1,"광해, 왕이 된 남자",2012-09-13,"88,899,448,769",3.30%,"88,899,448,769","12,319,390","12,319,390","1,001","203,400",한국,"리얼라이즈픽쳐스(주),(주)씨제이이엔엠",(주)씨제이이엔엠,15세이상관람가,사극,추창민,"이병헌,류승룡,한효주,장광,김인권,심은경,김명곤,서진원,장재현,정창국,조혜정,김남준..."
2,트랜스포머 3,2011-06-29,"74,840,681,500",2.80%,"74,840,681,500","7,784,743","7,784,743","1,409","124,450",미국,NaN,(주)씨제이이엔엠,12세이상관람가,액션,마이클 베이,"조쉬 더하멜,존 말코비치,로지 헌팅턴-휘틀리,샤이아 라보프,패트릭 뎀시"
3,미션임파서블:고스트프로토콜,2011-12-15,"57,301,924,000",2.10%,"57,301,924,000","7,508,896","7,508,896","1,024","114,765",미국,파라마운트 픽쳐스,(주)씨제이이엔엠,15세이상관람가,액션,브래드 버드,"톰 크루즈,제레미 레너"
4,최종병기 활,2011-08-10,"55,827,861,500",2.10%,"55,827,861,500","7,470,633","7,470,633",615,"130,054",한국,"(주)디씨지플러스,(주)다세포클럽",롯데쇼핑㈜롯데엔터테인먼트,15세이상관람가,사극,김한민,"박해일,류승룡,김무열,문채원,이한위,김구택,이경영,박기웅,이승준,이재구,박노식,이다..."


#### 파일 내보내기

In [149]:
df.to_csv("data_ver1.csv",encoding = 'utf-8-sig')